# Taller 5 — MLP_inicial.ipynb
### Autores: Sebastián Amortegui y Edwin

Notebook para el Taller 5 con regresión lineal, MLP (Grid Search y Random Search), ejercicio de hoja y evidencia.

In [ ]:
# %% 1. IMPORTS Y CONFIG
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from scipy.stats import uniform

RND = 42
print("Configuración lista.")

In [ ]:
# %% 2. CARGAR DATOS (Dataset colombiano ejemplo: residuos en Bogotá)
url = 'https://www.datos.gov.co/resource/ysq6-ri4e.csv'
df = pd.read_csv(url)
print('Dimensiones del dataset:', df.shape)
df.head()

In [ ]:
# %% 3. PREPARACIÓN
df = df.select_dtypes(include=['float64', 'int64']).dropna()
print('Columnas numéricas:', df.columns.tolist())
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RND)
print('Datos preparados.')

In [ ]:
# %% 4. REGRESIÓN LINEAL
pipe_lr = Pipeline([('scaler', StandardScaler()), ('lr', LinearRegression())])
pipe_lr.fit(X_train, y_train)
y_pred_lr = pipe_lr.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)
print(f'Regresión Lineal — MSE: {mse_lr:.4f}, R2: {r2_lr:.4f}')

In [ ]:
# %% 5. MLP con GRID SEARCH
pipe_mlp = Pipeline([('scaler', StandardScaler()), ('mlp', MLPRegressor(random_state=RND, max_iter=500))])
param_grid = {'mlp__hidden_layer_sizes': [(50,), (100,), (50,50)],
              'mlp__activation': ['relu', 'tanh'],
              'mlp__alpha': [0.0001, 0.001, 0.01],
              'mlp__learning_rate_init': [0.001, 0.01]}
gs = GridSearchCV(pipe_mlp, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
gs.fit(X_train, y_train)
y_pred_gs = gs.predict(X_test)
mse_gs = mean_squared_error(y_test, y_pred_gs)
r2_gs = r2_score(y_test, y_pred_gs)
print('MLP GridSearch — MSE: {:.4f}, R2: {:.4f}'.format(mse_gs, r2_gs))
print('Mejores parámetros (GridSearch):', gs.best_params_)

In [ ]:
# %% 6. MLP con RANDOM SEARCH
param_dist = {'mlp__hidden_layer_sizes': [(30,), (50,), (100,), (50,30)],
              'mlp__activation': ['relu', 'tanh'],
              'mlp__alpha': uniform(1e-5, 1e-2),
              'mlp__learning_rate_init': uniform(1e-4, 1e-2)}
rs = RandomizedSearchCV(pipe_mlp, param_dist, n_iter=10, cv=3, scoring='neg_mean_squared_error',
                        n_jobs=-1, random_state=RND, verbose=1)
rs.fit(X_train, y_train)
y_pred_rs = rs.predict(X_test)
mse_rs = mean_squared_error(y_test, y_pred_rs)
r2_rs = r2_score(y_test, y_pred_rs)
print('MLP RandomSearch — MSE: {:.4f}, R2: {:.4f}'.format(mse_rs, r2_rs))
print('Mejores parámetros (RandomSearch):', rs.best_params_)

In [ ]:
# %% 7. GUARDAR MODELOS
joblib.dump(pipe_lr, 'model_linear.pkl')
joblib.dump(gs.best_estimator_, 'mlp_grid_best.pkl')
joblib.dump(rs.best_estimator_, 'mlp_random_best.pkl')
print('Modelos guardados.')

# Respuestas (guía para completar)

- El modelo MLP con GridSearch tuvo un mejor desempeño que la regresión lineal.

- La métrica R² más alta indica un mejor ajuste del modelo.

- Los hiperparámetros clave fueron el tamaño de las capas y el valor de alpha.

- No se evidenció sobreajuste significativo si el R² del test fue similar al del entrenamiento.

- En general, el MLP capturó relaciones no lineales que la regresión no modeló bien.

In [ ]:
# %% 8. EJERCICIO DE HOJA
W1 = np.array([[0.1,0.2,0.1],[-0.2,0.1,0.2],[0.2,0.3,-0.1]])
b1 = np.array([0.1,0.2,0.1])
W2 = np.array([0.1,0.2,-0.1])
b2 = 0.1

def sigmoid(z): return 1/(1+np.exp(-z))

x = np.array([2,1,1])
y_true = 0
z1 = W1.dot(x) + b1
a1 = sigmoid(z1)
z2 = W2.dot(a1) + b2
a2 = sigmoid(z2)
error = 0.5*(y_true - a2)**2

print('z1 =', z1)
print('a1 =', a1)
print('z2 =', z2)
print('Salida final a2 =', a2)
print('Error =', error)

# Evidencia de congreso

**Integrantes:** Sebastián Amortegui, Edwin

Incluyan su foto y escriban su reflexión de 5 líneas sobre la charla del congreso.